# Vectorstores and Embeddings
Recall the overall workflow for retrieval augmented generation (RAG):

In [13]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

We just discussed Document Loading and Splitting.

In [14]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [15]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [16]:
splits = text_splitter.split_documents(docs)

In [17]:
len(splits)

209

## Embeddings
Let's take our splits and embed them.

In [18]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [19]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [20]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [25]:
len(embedding1), len(embedding2), len(embedding3)

(1536, 1536, 1536)

In [26]:
embedding1[:10]

[-0.027500716969370842,
 -0.00541996443644166,
 -0.025720367208123207,
 -0.03313217684626579,
 -0.027298690751194954,
 0.022487958893179893,
 -0.010334865190088749,
 -0.00820097140967846,
 0.002463763579726219,
 -0.019849002361297607]

In [21]:
import numpy as np

In [27]:
np.dot(embedding1, embedding2)

0.9631853877103518

In [28]:
np.dot(embedding1, embedding3)

0.7709997651294672

In [29]:
np.dot(embedding2, embedding3)

0.7596334120325523

## Vectorstores

In [31]:
# ! pip install chromadb

In [32]:
from langchain.vectorstores import Chroma

In [33]:
persist_directory = 'docs/chroma/'

In [34]:
!rm -rf ./docs/chroma  # remove old database files if any

In [35]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [36]:
print(vectordb._collection.count())

209


In [37]:
len(splits)

209

## Similarity Search

In [38]:
question = "is there an email i can ask for help"

In [39]:
docs = vectordb.similarity_search(question,k=3) # k is number of documents

In [40]:
len(docs)

3

In [41]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to succeed and \ndo well in this class and even help you to enjoy this cla ss more is if you form a study \ngroup.  \nSo start looking around where you' re sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to f

In [46]:
# persist vectorDB to use in future 

vectordb.persist()

## Failure modes
This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up.

Here are some edge cases that can arise - we'll fix them in the next class.

### Edge Case 1

In [52]:
question = "what did they say about matlab?"

In [48]:
docs = vectordb.similarity_search(question,k=5)

Notice that we're getting duplicate chunks (because of the duplicate MachineLearning-Lecture01.pdf in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

docs[0] and docs[1] are indentical.

In [50]:
docs[0]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

In [51]:
docs[1]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

### Edge Case 2

We can see a new failure mode.

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [53]:
question = "what did they say about regression in the third lecture?"

In [57]:
docs = vectordb.similarity_search(question,k=3)

In [58]:
for doc in docs:
    print(doc.metadata)

{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 0}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 14}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture02.pdf', 'page': 0}


In [66]:
print(docs[0].page_content)

MachineLearning-Lecture03  
Instructor (Andrew Ng) :Okay. Good morning and welcome b ack to the third lecture of 
this class. So here’s what I want to do t oday, and some of the topics I do today may seem 
a little bit like I’m jumping, sort  of, from topic to topic, but here’s, sort of, the outline for 
today and the illogical flow of ideas. In the last lecture, we  talked about linear regression 
and today I want to talk about sort of an  adaptation of that called locally weighted 
regression. It’s very a popular  algorithm that’s actually one of my former mentors 
probably favorite machine learning algorithm.  
We’ll then talk about a probabl e second interpretation of linear regression and use that to 
move onto our first classification algorithm, which is logistic regr ession; take a brief 
digression to tell you about something cal led the perceptron algorithm, which is 
something we’ll come back to, again, later this  quarter; and time allowing I hope to get to 
Newton’s method,

Approaches discussed in the next lecture can be used to address both!